In [1]:
import pandas as pd

DATA = '/kaggle/input/crime-data-from-2020-to-present/Crime_Data_from_2020_to_Present.csv'
df = pd.read_csv(filepath_or_buffer=DATA, )
df['Reported'] = pd.to_datetime(df['Date Rptd'].apply(func=lambda x: x.split()[0]))
df['Occurred'] = pd.to_datetime(df['DATE OCC'].apply(func=lambda x: x.split()[0]))
df['year_occurred'] = df['Occurred'].dt.year
df.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,Reported,Occurred,year_occurred
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506,2020-03-01,2020-03-01,2020
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628,2020-02-09,2020-02-08,2020
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002,2020-11-11,2020-11-04,2020
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387,2023-05-10,2020-03-10,2020
4,200412582,09/09/2020 12:00:00 AM,09/09/2020 12:00:00 AM,630,4,Hollenbeck,413,1,510,VEHICLE - STOLEN,...,NaN,NaN,NaN,200 E AVENUE 28,NaN,34.0820,-118.2130,2020-09-09,2020-09-09,2020


In [2]:
df.shape

(1004847, 31)

In [3]:
df['year_occurred'].value_counts().to_frame().T

year_occurred,2022,2023,2021,2020,2024
count,235252,232353,209871,199846,127525


In [4]:
sample_df = df[(df['year_occurred'] == 2024) & (df['LAT'] != 0)].drop(columns=['year_occurred', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'Cross Street', 'Weapon Used Cd', 'Weapon Desc'])

In [5]:
sample_df.dropna().shape

(97452, 24)

We have way too many datapoints to plot them all, even after we've reduced our focus to just one year and remove some broken records. So let's just plot a sample.

In [6]:
from plotly import colors
from plotly import express
from plotly import io

N = 3000
RANDOM_STATE = 2025

io.renderers.default = 'iframe'
express.scatter_mapbox(mapbox_style='open-street-map', height=800, zoom=9.25, data_frame=sample_df.sample(n=N, random_state=RANDOM_STATE),
                       lat='LAT', lon='LON', color='AREA NAME', color_discrete_sequence=colors.sample_colorscale('HSV', 21))

We don't expect crime to be evenly distributed across our areas, but do we have an expectation regarding how it is distributed?

In [7]:
sample_df['AREA NAME'].value_counts(normalize=True).to_frame().head(n=15).T

AREA NAME,Central,Southwest,Pacific,N Hollywood,77th Street,Wilshire,Hollywood,Devonshire,Olympic,Van Nuys,Southeast,Rampart,Topanga,West LA,Newton
proportion,0.080094,0.065109,0.063901,0.056836,0.053127,0.049402,0.048273,0.046109,0.045238,0.045144,0.043929,0.043913,0.043435,0.042423,0.042243


Not surprisingly crime is not evenly distributed; most areas in our top 15 areas hover around 4 1/2 percent.

Let's look at the top crime codes.

In [8]:
sample_df['Crm Cd Desc'].value_counts(normalize=True).to_frame().head(n=10).T

Crm Cd Desc,VEHICLE - STOLEN,BURGLARY FROM VEHICLE,SHOPLIFTING - PETTY THEFT ($950 & UNDER),THEFT PLAIN - PETTY ($950 & UNDER),"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),THEFT OF IDENTITY,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD",THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER),TRESPASSING
proportion,0.168665,0.07466,0.070935,0.069139,0.062294,0.059659,0.049551,0.047042,0.044054,0.043733


Wow. A lot of our top crime codes are vehicle-related.